<a href="https://colab.research.google.com/github/HausReport/MightyLogic/blob/master/Notebooks/Event_Chest_Sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up environment


In [1]:

import sys
# need newer plotly/plotly_express than collab provides?
#!{sys.executable} -m pip install --force-reinstall --upgrade "plotly"
#!{sys.executable} -m pip install --force-reinstall --upgrade "plotly_express"

#
# Moving some of the guts to GitHub and refactoring 
#
!{sys.executable} -m pip install --upgrade --force-reinstall "git+https://github.com/HausReport/MightyLogic.git#egg=MightyLogic"

#
# Imports
#
from random import randrange
from scipy.signal import convolve2d
from enum import Enum, auto
from pprint import pprint
import plotly.figure_factory as ff

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import math

#import sklearn
#from sklearn.preprocessing import PowerTransformer, QuantileTransformer

  Cloning https://github.com/HausReport/MightyLogic.git to /tmp/pip-install-t61alarh/mightylogic_3f1a8893d9444a45aa9aaac72c315f2f
  Running command git clone -q https://github.com/HausReport/MightyLogic.git /tmp/pip-install-t61alarh/mightylogic_3f1a8893d9444a45aa9aaac72c315f2f
     |████████████████████████████████| 280 kB 34.7 MB/s 
     |████████████████████████████████| 53 kB 2.4 MB/s 
     |████████████████████████████████| 40 kB 4.0 MB/s 
     |████████████████████████████████| 97 kB 7.5 MB/s 
     |████████████████████████████████| 67 kB 6.5 MB/s 
  Created wheel for MightyLogic: filename=MightyLogic-0.2-py3-none-any.whl size=26196 sha256=fe832550c72b0573ab8dbe3741fe56952de9f1f7846225c063b8d572d0fc2ffe
  Stored in directory: /tmp/pip-ephem-wheel-cache-7jx7dpnz/wheels/84/7a/55/d74e49b550ce0fddb2379db726853920650df706dedea8daae
Successfully built MightyLogic
  Attempting uninstall: zipp
    Found existing installation: zipp 3.5.0
    Uninstalling zipp-3.5.0:
      Successfully unin

In [2]:

from MightyLogic.Currency import Currency as Currency
from MightyLogic.Values import Values as Values

Values.GOLD

0.6

# Simulate Buying Event Chests

In [3]:
from MightyLogic.Rewards.RewardFactory import rare_chest, epic_chest , legendary_chest

#
# Simulate spending 5000 sparks on various types of event chests
#

def k_sim(spx, cost, verbose=False):
  df = pd.DataFrame()
  tot = 0
  sparks = spx
  rolls = 0
  spent = 0 
  while sparks>(cost-1):
    ret = None
    if cost==250:
      ret = rare_chest()
    elif cost==1000:
      ret = epic_chest()
    elif cost==5000:
      ret = legendary_chest()
    rolls +=1
    tot += ret.myValue
    df = df.append(ret.toDict(),ignore_index=True)
    sparks = sparks + ret.SPARK
    spent = spent+cost
    sparks = sparks - cost


  cost = 5000 * Values.SPARK
  eff = tot/cost

  if verbose:  
    print(f"Rolls: {rolls}")
    print(f"Total value: {tot}")
    print(f"Total efficiency: {eff}")
    print(f"Spent: {spent}")

  return df, rolls, spent  

def fivek_sim(cost, verbose=False):
  return k_sim(5000, cost, verbose)

def fivek_rare(verbose=False):
  return fivek_sim(250, verbose=verbose)

def fivek_legend(verbose=False):
  return fivek_sim(5000,verbose=verbose)

def fivek_epic(verbose=False):
  return fivek_sim(1000, verbose=verbose)

In [4]:
#
#Total value gained spending 5000 sparks on epic chests
#
df, rolls, spent = fivek_epic(verbose=True)
df['value'].sum()

Rolls: 5
Total value: 162.3
Total efficiency: 0.001623
Spent: 5000


162.3

In [5]:
#
# Simulate spending 5000 sparks on epic chests 1000 times and show resulting value obtained
#
results = []
for i in range(0,1000):
  df, rolls, spent = fivek_epic()
  results.append(df['value'].sum())

#pprint(results)
fig = px.histogram(pd.DataFrame(results,columns=["Value"]),x="Value")
fig.show()

In [6]:
#
# Simulate spending 5000 sparks on epic chests 1000 times and show resulting number of rolls possible
#
results = []
for i in range(0,1000):
  df, rolls, spent = fivek_epic()
  results.append(rolls)

#pprint(results)
df = pd.DataFrame(results,columns=["Rolls"])
fig = px.histogram(df,x="Rolls")
fig.show()

In [7]:
#
# Above, but tabular format
#
import numpy as np

df.groupby(['Rolls'], as_index=False).size()
#.aggregate(np.sum)

,Rolls,size
0,5,398
1,6,292
2,7,107
3,8,77
4,9,54
5,10,28
6,11,20
7,12,13
8,13,2
9,14,3


In [37]:
#
# Simulate spending 5000 sparks on legendary chests 1000 times and show resulting number of rolls possible
#
results = []
for i in range(0,1000):
  df, rolls, spent = fivek_legend()
  results.append(rolls)

#pprint(results)
df = pd.DataFrame(results,columns=["Rolls"])
fig = px.histogram(df,x="Rolls", range_x=[0,14]) #, log_y=True, xmin=0, xmax=100)
fig.update_layout(title='# Rolls From Spending 5000 sparks on Legendary Chests', autosize=False,
                  width=600, height=500, 
                  margin=dict(l=65, r=50, b=65, t=90))


print("Tier II (League 10-14, Chapter 3 of Event")
fig.show()

Tier II (League 10-14, Chapter 3 of Event


In [43]:
#
# Above, but tabular format
#
import numpy as np

foo = df.groupby(['Rolls'], as_index=False).size()
foo['percent'] = foo['size']/10
foo['sparks_spent'] = foo['Rolls']*5000
#.aggregate(np.sum)
print("Tier II (League 10-14, Chapter 3 of Event")
foo

Tier II (League 10-14, Chapter 3 of Event


,Rolls,size,percent,sparks_spent
0,1,724,72.4,5000
1,2,182,18.2,10000
2,3,37,3.7,15000
3,4,26,2.6,20000
4,5,12,1.2,25000
5,6,2,0.2,30000
6,7,4,0.4,35000
7,8,7,0.7,40000
8,9,2,0.2,45000
9,10,3,0.3,50000


In [32]:
#
# Simulate spending 5000 sparks on epic chests 1000 times and show resulting number of rolls possible
#
results = []
for i in range(0,1000):
  df, rolls, spent = fivek_epic()
  results.append(rolls)

#pprint(results)
df = pd.DataFrame(results,columns=["Rolls"])
fig = px.histogram(df,x="Rolls") #, log_y=True)
fig.update_layout(title='# Rolls From Spending 5000 sparks on Epic Chests', autosize=False,
                  width=600, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [11]:
#
# Simulate spending 5000 sparks on rare chests 1000 times and show resulting number of rolls possible
#
results = []
for i in range(0,1000):
  df, rolls, spent = fivek_rare()
  results.append(rolls)

#pprint(results)
df = pd.DataFrame(results,columns=["Rolls"])
fig = px.histogram(df,x="Rolls") #, log_y=True)
fig.update_layout(title='# Rolls From Spending 5000 sparks on Rare Chests', autosize=False,
                  width=600, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [12]:
fivek_rare()

(    COMMON_SOUL  CONTRIBUTION  EPIC_SOUL  ...  SOUL_DUST  SPARK    value
 0           0.0           0.0        0.0  ...        0.0    0.0     1.10
 1           0.0           0.0        0.0  ...        0.0    0.0     0.81
 2           0.0           0.0        0.0  ...        0.0    0.0     1.92
 3           0.0           0.0        0.0  ...        0.0  226.0  4520.00
 4           0.0           0.0        0.0  ...        0.0    0.0     1.65
 5           0.0           0.0        0.0  ...        0.0    0.0     1.73
 6           0.0           0.0        0.0  ...        0.0    0.0     1.31
 7           0.0           0.0        0.0  ...        0.0    0.0     1.41
 8           0.0           0.0        0.0  ...        0.0    0.0     1.19
 9           0.0           0.0        0.0  ...        0.0    0.0     0.88
 10          0.0           0.0        0.0  ...        0.0    0.0     1.63
 11          0.0           0.0        0.0  ...        0.0    0.0  5580.00
 12          0.0           0.0        

In [13]:
fivek_legend()

(   COMMON_SOUL  CONTRIBUTION  EPIC_SOUL  ...  SOUL_DUST   SPARK     value
 0          0.0           0.0        0.0  ...        0.0  9529.0  190580.0
 1          0.0           0.0        0.0  ...        0.0     0.0     174.0
 
 [2 rows x 11 columns], 2, 10000)

In [14]:
#
# Simulate spending 1000 sparks on epic chests 1000 times and show resulting value obtained
#
results = []
for i in range(0,1000):
  df, rolls, spent = k_sim(1000,1000)
  results.append(df['value'].sum())

#pprint(results)
fig = px.histogram(pd.DataFrame(results,columns=["Value"]),x="Value")
fig.show()

In [15]:
#
# Simulate spending 1000 sparks on epic chests 1000 times and show resulting value obtained
#
results = []
for i in range(0,1000):
  df, rolls, spent = k_sim(1000,250)
  results.append(df['value'].sum())

#pprint(results)
fig = px.histogram(pd.DataFrame(results,columns=["Value"]),x="Value")
fig.show()